## Imports e constantes

In [1]:
%matplotlib qt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import project_utils
import matplotlib.pyplot as plt
from itertools import product
from pprint import pprint
from signals import Signals
from ied import Ied

# Constantes dos transformadores de instrumentação
RTC = 300 / 5
RTPC = 500e3 / 115

# Constantes para o processo de downsampling
FUNDAMENTAL_PERIOD = 1 / 60
DESIRED_SAMPLE_RATE = 16

# Constantes do filtro de anti-aliasing
B = 1.599e3
C = 1.279e6

# Constantes dos relés de sobrecorrente
PHASE_TIMED_ADJUST_CURRENT = 3
PHASE_TIMED_GAMMA = {'a': 0.148, 'b': 0.069, 'c': 0.001, 'b\'': 0.001, 'c\'': 0.173, 'd\'': 0.375}
PHASE_INSTA_ADJUST_CURRENT = {'a': 6.05, 'b': 5.49, 'c': 5.03,  'b\'': 11.68, 'c\'': 14.71, 'd\'': 19.87}
NEUTRAL_TIMED_ADJUST_CURRENT = 0.5
NEUTRAL_TIMED_GAMMA = {'a':0.494, 'b': 0.240, 'c': 0.001, 'b\'': 0.001, 'c\'': 0.312, 'd\'': 0.658}
NEUTRAL_INSTA_ADJUST_CURRENT = {'a': 1.21, 'b': 1.10, 'c': 1.01, 'b\'': 2.34, 'c\'': 2.94, 'd\'': 3.97}

# Constantes para o relé 32
ALPHA = 90
BETA = 30

# Impedâncias da linha de transmissão
R1 = 0.0246 * 250
XL1 = 0.3219 * 250
R0 = 0.376 * 250
XL0 = 1.411 * 250
Z1 = R1 + 1j * XL1
Z0 = R0 + 1j * XL0

# Constante para o relé 21 com comparador cosseno
INCLINATION_ANGLE = np.angle(Z1)
ZONES_IMPEDANCES = {
    "zone1": np.abs(0.85 * Z1),
    "zone2": np.abs(1.5 * Z1),
    "zone3": np.abs(2 * Z1),
}

# Características moh para plot das zonas de proteção
ZONE1C = [0.1247, 0.4654, 0.9636]
ZONE2C = [0.2200, 0.8212, 1.7004]
ZONE3C = [0.2934, 1.0950, 2.2672]

## Carregamento dos dados

In [2]:
data = {
    f'bus{bus}_fault{fault}': pd.read_csv(f"Registros/Registro{fault}/1Reg{bus}.dat", delimiter='\\s+',
                                              names=['step', 't', 'Va', 'Vb', 'Vc', 'Ia', 'Ib', 'Ic'])
    for bus, fault in product((1, 2), (1, 2))
}

## Primerio estágio: Plot dos diferentes sinais

In [ ]:

# Exemplo para os sinais na barra 2, falta 1 (bus1 = barra 2)
sampling_period = data[f'bus{1}_fault{1}']['t'][1] - data[f'bus{1}_fault{1}']['t'][0]
md = int(FUNDAMENTAL_PERIOD / (DESIRED_SAMPLE_RATE * sampling_period))

signals = Signals(
        va=data[f'bus{1}_fault{1}']['Va'],  # type: ignore
        vb=data[f'bus{1}_fault{1}']['Vb'],  # type: ignore
        vc=data[f'bus{1}_fault{1}']['Vc'],  # type: ignore
        ia=data[f'bus{1}_fault{1}']['Ia'],  # type: ignore
        ib=data[f'bus{1}_fault{1}']['Ib'],  # type: ignore
        ic=data[f'bus{1}_fault{1}']['Ic'],  # type: ignore
        t=data[f'bus{1}_fault{1}']['t'],  # type: ignore
        sampling_period=sampling_period
    )

ied = Ied(signals=signals, b=B, c=C, R=R1, XL=XL1, samples_per_cycle=16, md=md)

figure, ax = plt.subplots(2, 1, figsize=(10, 10))
ax[0].set_title(f'Sinais coletados na barra 2 relativo à primeira falta')
ax[0].plot(ied._signals['t'], ied._signals['ia'], label=r'$i_a[n]$')
ax[0].plot(ied._aa_signals['t'], ied._aa_signals['ia'], label=r'$i_a[n]$ (AA)')
ax[0].scatter(ied._resampled_aa_signals['t'], ied._resampled_aa_signals['ia'], label=r'$i_a[n]$ (AA resampled)', s=10)
ax[0].scatter(ied._mimic_filtered_signals['t'], ied._mimic_filtered_signals['ia'], label=r'$i_a[n]$ (Mimic filtered)', s=10)
ax[0].plot(ied._mimic_filtered_signals['t'], np.abs(ied.phasors['ia']), label=r'$|\dot{I}_a|$')
ax[0].legend(fontsize='small')

ax[1].plot(ied._mimic_filtered_signals['t'], np.angle(ied.phasors['ia'], deg=True), label=r'$\angle{\dot{I}_a}$')  # type: ignore
ax[1].legend(fontsize='small')

plt.show(block=True)

## Segundo estágio: Adição das funções de proteção de sobrecorrente

### Coordenogramas

In [4]:

project_utils.plot_trip_curves(
    relays=['a', 'b', 'c'],
    timed_adjust_current=PHASE_TIMED_ADJUST_CURRENT,
    insta_adjust_current=PHASE_INSTA_ADJUST_CURRENT,
    gamma=PHASE_TIMED_GAMMA,
    title='Curvas de atuação dos relés de sobrecorrente fase'
)

project_utils.plot_trip_curves(
    relays=['a', 'b', 'c'],
    timed_adjust_current=NEUTRAL_TIMED_ADJUST_CURRENT,
    insta_adjust_current=NEUTRAL_INSTA_ADJUST_CURRENT,
    gamma=NEUTRAL_TIMED_GAMMA,
    title='Curvas de atuação dos relés de sobrecorrente neutro'
)

project_utils.plot_trip_curves(
    relays=["b'", "c'", "d'"],
    timed_adjust_current=PHASE_TIMED_ADJUST_CURRENT,
    insta_adjust_current=PHASE_INSTA_ADJUST_CURRENT,
    gamma=PHASE_TIMED_GAMMA,
    title='Curvas de atuação dos relés de sobrecorrente fase'
)

project_utils.plot_trip_curves(
    relays=["b'", "c'", "d'"],
    timed_adjust_current=NEUTRAL_TIMED_ADJUST_CURRENT,
    insta_adjust_current=NEUTRAL_INSTA_ADJUST_CURRENT,
    gamma=NEUTRAL_TIMED_GAMMA,
    title='Curvas de atuação dos relés de sobrecorrente neutro'
)

/home/andrey/Documentos/Code/sistemas_eletricos/project_utils.py:218: RuntimeWarning: divide by zero encountered in divide
  curve_common_term = gamma * (k / ((currents / timed_adjust_current) ** a - 1) + c)


### Diagrama polar das regiões do relé 32

In [5]:
project_utils.plot_32_polar_regions(
    v_pol_angles=[-85.58, 154.42, 34.42],
    i_op_angles=[-173.85, 66.15, -53.85],
    beta=BETA,
)

### Adição das funções

In [6]:
for fault, bus in product((1, 2), (1, 2)):
    sampling_period = data[f'bus{bus}_fault{fault}']['t'][1] - data[f'bus{bus}_fault{fault}']['t'][0]
    md = int(FUNDAMENTAL_PERIOD / (DESIRED_SAMPLE_RATE * sampling_period))

    signals = Signals(
        va=data[f'bus{bus}_fault{fault}']['Va'],  # type: ignore
        vb=data[f'bus{bus}_fault{fault}']['Vb'],  # type: ignore
        vc=data[f'bus{bus}_fault{fault}']['Vc'],  # type: ignore
        ia=data[f'bus{bus}_fault{fault}']['Ia'],  # type: ignore
        ib=data[f'bus{bus}_fault{fault}']['Ib'],  # type: ignore
        ic=data[f'bus{bus}_fault{fault}']['Ic'],  # type: ignore
        t=data[f'bus{bus}_fault{fault}']['t'],  # type: ignore
        sampling_period=sampling_period
    )

    ieds = {
        name: Ied(signals=signals, b=B, c=C, md=md, R=R1, XL=XL1, samples_per_cycle=16)
        for name in ['b', 'c\'']
    }

    for ied_name, ied in ieds.items():
        ied.add_relay(
            relay_type='51F',
            gamma=PHASE_TIMED_GAMMA[ied_name],
            adjust_current=PHASE_TIMED_ADJUST_CURRENT,
            curve='IEEE_moderately_inverse',
        )
        ied.add_relay(
            relay_type='51N',
            gamma=NEUTRAL_TIMED_GAMMA[ied_name],
            adjust_current=NEUTRAL_TIMED_ADJUST_CURRENT,
            curve='IEEE_moderately_inverse',
        )
        ied.add_relay(
            relay_type='50F',
            adjust_current=PHASE_INSTA_ADJUST_CURRENT[ied_name],
        )
        ied.add_relay(
            relay_type='50N',
            adjust_current=NEUTRAL_INSTA_ADJUST_CURRENT[ied_name],
        )
        ied.add_relay(
            relay_type="32F",
            alpha = ALPHA,
            beta = BETA,
        )
        ied.add_relay(
            relay_type="32N",
            alpha = ALPHA,
            beta = BETA,
        )
        ied.add_relay(
            relay_type="67F",
        )
        ied.add_relay(
            relay_type="67N",
        )

    if bus == 1:
        project_utils.plot_51f_50f_32f_trips(ieds['b'], f'Bus {bus + 1} Fault {fault}')
        project_utils.plot_51n_50n_32n_67f_67n_trips(ieds['b'], f'Bus {bus + 1} Fault {fault}')
    else:
        project_utils.plot_51f_50f_32f_trips(ieds['c\''], f'Bus {bus + 1} Fault {fault}')
        project_utils.plot_51n_50n_32n_67f_67n_trips(ieds['c\''], f'Bus {bus + 1} Fault {fault}')



## Terceiro estágio: Função 21 (medida direta)

In [7]:
with open("impedancias_medidas.txt", "w") as f:
    pass

for bus, fault in product((1, 2), (1, 2)):
    signals = Signals(
        va=data[f'bus{bus}_fault{fault}']['Va'],  # type: ignore
        vb=data[f'bus{bus}_fault{fault}']['Vb'],  # type: ignore
        vc=data[f'bus{bus}_fault{fault}']['Vc'],  # type: ignore
        ia=data[f'bus{bus}_fault{fault}']['Ia'],  # type: ignore
        ib=data[f'bus{bus}_fault{fault}']['Ib'],  # type: ignore
        ic=data[f'bus{bus}_fault{fault}']['Ic'],  # type: ignore
        t=data[f'bus{bus}_fault{fault}']['t'],  # type: ignore
        sampling_period=sampling_period
    )

    ieds = {
        name: Ied(signals=signals, b=B, c=C, md=md, R=R1, XL=XL1, samples_per_cycle=16)
        for name in ['b', 'c\'']
    }

    for ied_name, ied in ieds.items():
        ied.add_relay(
            relay_type='21',
            inclination_angle=INCLINATION_ANGLE,
            line_positive_seq_impedance=Z1,
            line_zero_seq_impedance=Z0,
            abs_zones_impedances = ZONES_IMPEDANCES,
        )

    curr_ied = ieds['b'] if bus == 1 else ieds['c\'']
    for unit in ['at', 'bt', 'ct', 'ab', 'bc', 'ca']:
        primary_reflected_impedance = curr_ied._trips['21']['measured_impedances'][unit][-1] * (RTPC / RTC)
        
        with open("impedancias_medidas.txt", "a") as f:
            f.write(f'*Falta {"ACT" if fault == 1 else "BT"} na barra {bus + 1}*\n')
            f.write(f'Impedância vista pela unidade {unit.upper()} na barra {bus + 1}: {np.round(curr_ied._trips["21"]["measured_impedances"][unit][-1], 2)} ohms\n')
            f.write(f'Impedância calculada ao ponto da falta a partir da unidade {unit.upper()} na barra {bus + 1}: {np.round(primary_reflected_impedance, 2)} ohms\n')
            f.write(f'Distância da falta observada pela unidade {unit.upper()} na barra {bus + 1}: {np.round(np.abs(primary_reflected_impedance) / np.abs(0.0246 + 1j * 0.3219), 2)} km\n')
            f.write(f'Resultado válido ? {"*Sim*" if np.abs(primary_reflected_impedance) <= np.abs(2*Z1) else "*Não*"}\n')
            f.write('\n')

    with open(f"bus{bus}_fault{fault}_21.txt", 'w') as f:
        f.write(f'*Falta {"ACT" if fault == 1 else "BT"} na barra {bus + 1}*\n')
        pprint(curr_ied._trips["21"]["trip_signals"], stream=f)